# 7.16 Air Lines

Air at 65 psig and 115 deg F flows through 75 feet of 1" schedule 40 pipe at a rate of 100 cubic feet/minute.

Find the pressure drop in psi and the velocity in feet per minute at the inlet and the outlet.

In [1]:
from fluids.units import *
from math import pi
P1 = 65*u.psi + 1*u.standard_atmosphere

In [2]:
# Take nitrogen-oxygen as air
from thermo import PRMIX
zs = [0.79, 0.21]
Tcs = [126.2, 154.58]
Pcs = [3394387.5, 5042945.25]
omegas = [0.04, 0.021]

eos_flowing = PRMIX(T=(115*u.degF).to(u.K).magnitude, P=P1.to(u.Pa).magnitude, zs=zs, Tcs=Tcs, Pcs=Pcs, omegas=omegas)
eos_std = PRMIX(T=288.15, P=101325.0, zs=zs, Tcs=Tcs, Pcs=Pcs, omegas=omegas)
V_ratio = eos_flowing.V_g/eos_std.V_g
print('Ratio of actual to standard flow: %s' %(V_ratio))

Ratio of actual to standard flow: 0.20407676712438672


In [3]:
NPS, Di, Do, t = nearest_pipe(NPS=1, schedule='40')
Q = V_ratio*100*u.ft**3/u.min
L = 75*u.ft

MW = 28.958*u.g/u.mol
Vm = eos_flowing.V_g*u.m**3/u.mol
rho = (Vm)**-1*MW

mu = 1.93e-5*u.Pa

A = 0.25*pi*Di**2

v0 = Q/A
m = v0*A*rho

Re = rho*v0*Di/mu
fd = friction_factor(Re=Re, eD=0.0018*u.inch/Di)
P2 = isothermal_gas(rho=rho, fd=fd, P1=P1, P2=None, L=L, D=Di, m=m)

print('pressure drop = %s' %(P1-P2))
print('upstream speed = %s' %(v0.to(u.ft/u.min)))

pressure drop = 2.714842967501866 pound_force_per_square_inch
upstream speed = 3401.458438219272 foot / minute


In [4]:
eos_end = PRMIX(T=(115*u.degF).to(u.K).magnitude, P=P2.to(u.Pa).magnitude, zs=zs, Tcs=Tcs, Pcs=Pcs, omegas=omegas)
Vm2 = eos_end.V_g*u.m**3/u.mol
rho2 = (Vm2)**-1*MW
print('downstream speed = %s' %((v0*rho/rho2).to(u.ft/u.min)))

downstream speed = 3521.622706075368 foot / minute


The answers given in Crane are 2.61 psi; and 3367 and 3484 ft/min respectively. A tabular method is used there of limited accuracy.

For compressible fluids, there is a benefit to breaking the problem up into sections and performing the calculation to each of them, preferably while including the JT effect and heat loss/gain.